# Dataset
```
a b c | a
x y z | x
1 2 3 | 1
```

In [150]:
import string
pool = string.ascii_letters + string.digits
pool_size = len(pool)
pool, pool_size

('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', 62)

In [151]:
stoi = {s:i for i,s in enumerate(pool)}
itos = {i:s for s,i in stoi.items()}
encode = lambda str: [stoi[c] for c in str]
decode = lambda ints: ''.join([itos[i] for i in ints])
x = encode('john')
x, decode(x)


([9, 14, 7, 13], 'john')

In [152]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [153]:
row_count = 800
col_count = 3
dataset = torch.randint(0, len(pool), (row_count,col_count))
X=dataset
#Y=dataset[:, 0].unsqueeze(dim=1)
Y=dataset[:, 0]

for row in dataset[:3]:
  print(decode(row.tolist()))

X.shape, X.dtype, Y.shape, Y.dtype

weV
C50
uxy


(torch.Size([800, 3]), torch.int64, torch.Size([800]), torch.int64)

In [154]:
X.shape, X[19]

(torch.Size([800, 3]), tensor([33, 36,  5]))

In [155]:
n_embd = 10
class SomeModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embeddings = nn.Embedding(pool_size, n_embd)
    self.l1 = nn.Linear(n_embd, pool_size)

  def forward(self, idx, targets=None):
    x = self.token_embeddings(idx) # (B,T,C)
    x = self.l1(x) # (B,T, pool_size)
    if targets is None:
      return x, None
    loss = F.cross_entropy(x[:,-1,:], targets)
    return x, loss
  
  def generate(self, idx):
    logits, _ = self(idx)
    probs = F.softmax(logits, dim=1)
    hit = torch.multinomial(probs[:,-1,:], num_samples=1)
    return hit

    


In [156]:
m = SomeModel()
optimizer = torch.optim.AdamW(params=m.parameters(), lr=0.1)


In [178]:
for _ in range(10):
  logits, loss=m(X, Y)
  print(loss)

  optimizer.zero_grad(set_to_none=None)
  loss.backward()
  optimizer.step()


given=torch.tensor(encode('abc')).unsqueeze(dim=0)
print(given.shape)
out=[]
for _ in range(10):
  hit = m.generate(given)[0].item()
  out.append(itos[hit])
print(f'answers (we expect a): {"".join(out)}')


given=torch.tensor(encode('834')).unsqueeze(dim=0)
print(given.shape)
out=[]
for _ in range(10):
  hit = m.generate(given)[0].item()
  out.append(itos[hit])
print(f'answers (we expect 8): {"".join(out)}')



tensor(2.5993, grad_fn=<NllLossBackward0>)
tensor(2.5984, grad_fn=<NllLossBackward0>)
tensor(2.5973, grad_fn=<NllLossBackward0>)
tensor(2.5966, grad_fn=<NllLossBackward0>)
tensor(2.5962, grad_fn=<NllLossBackward0>)
tensor(2.5961, grad_fn=<NllLossBackward0>)
tensor(2.5964, grad_fn=<NllLossBackward0>)
tensor(2.5967, grad_fn=<NllLossBackward0>)
tensor(2.5967, grad_fn=<NllLossBackward0>)
tensor(2.5962, grad_fn=<NllLossBackward0>)
torch.Size([1, 3])
answers (we expect a): vZh0m02nDZ
torch.Size([1, 3])
answers (we expect 8): krCCErrxXr


its not learning sequence/order because we don't have it in our model